# Tables

This script is to generate tables for our paper.

In [1]:
%load_ext autoreload
%autoreload 2

import json
from pathlib import Path

In [2]:
def get_metrics(method, resolution=8, image=True):
    """
    Get the metrics. If image==True, then use every even index for metrics to evaluate image generation. If image==False, then use every odd index.
    """
    filename = str(sorted(list(Path("/home/ethanjohnweber/monitor/eval---eval/" + method + "/transformer/").iterdir()))[-1]) + "/metrics/256.json"
    with open(filename, "r") as f:
        metrics_dict = json.load(f)
    indices = sorted([int(x) for x in metrics_dict.keys()])
    indices = indices[::2] if image else indices[1::2]

    psnrs = 0.0
    vals = 0.0
    ssims = 0.0
    lpipss = 0.0
    for i in indices:
        md = metrics_dict[str(i)][str(resolution)]
        psnr = md["image-psnr"] if image else (md["origins-psnr"] + md["ray-psnr"])
        psnr = md["image-psnr"] if image else (md["origins-psnr"] + md["ray-psnr"])
        ssim = md["image-ssim"] if image else (md["origins-ssim"] + md["ray-ssim"])
        lpips = md["image-lpips"] if image else (md["origins-lpips"] + md["ray-lpips"])

        val = 0.0
        num = 0
        for key, value in md["validation_losses"]["con-im" if image else "con-ra"].items():
            if key != "tensor(1000., device='cuda:0')" and key != "1000":
                val += value
                num += 1
        val /= num

        psnrs += psnr
        ssims += ssim
        lpipss += lpips
        vals += val

    metrics = [psnrs / len(indices), ssims / len(indices), lpipss / len(indices), vals / len(indices)]
    return metrics

In [3]:
METHODS = [
    ("no-index-embeddings", "no-index-emb"),
    ("fixed-index-embeddings", "fixed-index-emb"),
    ("no-pose-pred", "no-pose-pred"),
    ("random-pose-aug", "random-pose-aug"),
    ("train", "\\method{} (100K iters)"),
    ("final", "\\method{} (1M iters)"),
]
# 8-view, 16-view, 32-view
# psnr ssim lpips val | psnr ssim lpips val | psnr ssim lpips val
SIGFIGS = [2, 3, 3, 4, 2, 3, 3, 4, 2, 3, 3, 4]
def print_datasets():
    # create the experiment lines
    EXPERIMENT_LINES = ["\n"]
    for method, method_name in METHODS:
        metrics_8 = get_metrics(method, resolution=8)
        metrics_8 = [str(f"{metric:.{sigfigs}f}") for metric, sigfigs in zip(metrics_8, SIGFIGS)]

        metrics_16 = get_metrics(method, resolution=16)
        metrics_16 = [str(f"{metric:.{sigfigs}f}") for metric, sigfigs in zip(metrics_16, SIGFIGS)]

        metrics_32 = get_metrics(method, resolution=32)
        metrics_32 = [str(f"{metric:.{sigfigs}f}") for metric, sigfigs in zip(metrics_32, SIGFIGS)]

        line_string = f"{method_name} & "
        for i in range(len(metrics_8)):
            line_string += f"{metrics_8[i]}"
            line_string += ' & '
        for i in range(len(metrics_16)):
            line_string += f"{metrics_16[i]}"
            line_string += ' & '
        for i in range(len(metrics_32)):
            line_string += f"{metrics_32[i]}"
            if i != len(metrics_32) - 1:
                line_string += ' & '
        line_string += "\\\\ \n"

        EXPERIMENT_LINES.append(line_string)

    print(" ".join(EXPERIMENT_LINES))

In [5]:
print_datasets()


 no-index-emb & 11.10 & 0.431 & 0.456 & 0.2394 & 14.06 & 0.450 & 0.400 & 0.2417 & 12.38 & 0.467 & 0.422 & 0.2438\\ 
 fixed-index-emb & 10.46 & 0.353 & 0.520 & 0.2517 & 12.14 & 0.390 & 0.491 & 0.2546 & 12.52 & 0.416 & 0.448 & 0.2556\\ 
 no-pose-pred & 11.63 & 0.413 & 0.426 & 0.2386 & 14.73 & 0.461 & 0.384 & 0.2411 & 13.39 & 0.476 & 0.389 & 0.2431\\ 
 random-pose-aug & 11.82 & 0.431 & 0.415 & 0.2384 & 16.18 & 0.487 & 0.333 & 0.2409 & 14.27 & 0.483 & 0.367 & 0.2430\\ 
 \method{} (100K iters) & 11.97 & 0.435 & 0.415 & 0.2383 & 15.81 & 0.481 & 0.329 & 0.2407 & 14.21 & 0.486 & 0.366 & 0.2426\\ 
 \method{} (1M iters) & 12.77 & 0.442 & 0.381 & 0.2365 & 17.20 & 0.485 & 0.281 & 0.2388 & 14.13 & 0.498 & 0.352 & 0.2396\\ 

